In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import pandas_profiling as pp
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction

# Metodologia adotada para limpeza :
* Merge dos dados de treino e teste
* Excluir colunas
* Mediana dos valores de um array
* Normalização dos dados

In [0]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [0]:
print("Treino : " , data.shape)
print("Teste : " , data_test.shape)

Treino :  (5729, 44)
Teste :  (2848, 43)


In [0]:
data.head(3)

,ID,League,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,Year,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR,Result
0,GER310812#0,Germany,31/08/12,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,2013,[8.0],[3.0],[13.0],[8.0],[2.0],[0.0],[8.0],[3.0],[13.0],[8.0],[2.0],[0.0],0
1,GER010912#0,Germany,01/09/12,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,2013,[9.0],[4.0],[14.0],[0.0],[2.0],[0.0],[9.0],[4.0],[14.0],[0.0],[2.0],[0.0],1
2,GER010912#1,Germany,01/09/12,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,2013,[14.0],[8.0],[6.0],[4.0],[0.0],[0.0],[14.0],[8.0],[6.0],[4.0],[0.0],[0.0],0


In [0]:
data.head(3)

,ID,League,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,Year,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR,Result
0,GER310812#0,Germany,31/08/12,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,2013,[8.0],[3.0],[13.0],[8.0],[2.0],[0.0],[8.0],[3.0],[13.0],[8.0],[2.0],[0.0],0
1,GER010912#0,Germany,01/09/12,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,2013,[9.0],[4.0],[14.0],[0.0],[2.0],[0.0],[9.0],[4.0],[14.0],[0.0],[2.0],[0.0],1
2,GER010912#1,Germany,01/09/12,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,2013,[14.0],[8.0],[6.0],[4.0],[0.0],[0.0],[14.0],[8.0],[6.0],[4.0],[0.0],[0.0],0


#1.1 Limpeza dos dados

In [0]:
get_mean = lambda x : np.median(ast.literal_eval(x))
for column in ['HS', 'HST', 'HF', 'HC', 'HY', 'HR', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR']:
  data[column] = data[column].apply(get_mean)

data.head(3)

,ID,League,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,Year,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR,Result
0,GER310812#0,Germany,31/08/12,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,2013,8.0,3.0,13.0,8.0,2.0,0.0,8.0,3.0,13.0,8.0,2.0,0.0,0
1,GER010912#0,Germany,01/09/12,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,2013,9.0,4.0,14.0,0.0,2.0,0.0,9.0,4.0,14.0,0.0,2.0,0.0,1
2,GER010912#1,Germany,01/09/12,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,2013,14.0,8.0,6.0,4.0,0.0,0.0,14.0,8.0,6.0,4.0,0.0,0.0,0


In [0]:
get_mean = lambda x : np.median(ast.literal_eval(x))
for column in ['HS', 'HST', 'HF', 'HC', 'HY', 'HR', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR']:
  data_test[column] = data_test[column].apply(get_mean)

data.head(3)

,ID,League,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,Year,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR,Result
0,GER310812#0,Germany,31/08/12,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,2013,8.0,3.0,13.0,8.0,2.0,0.0,8.0,3.0,13.0,8.0,2.0,0.0,0
1,GER010912#0,Germany,01/09/12,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,2013,9.0,4.0,14.0,0.0,2.0,0.0,9.0,4.0,14.0,0.0,2.0,0.0,1
2,GER010912#1,Germany,01/09/12,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,2013,14.0,8.0,6.0,4.0,0.0,0.0,14.0,8.0,6.0,4.0,0.0,0.0,0


# Análise exploratória nos dados

In [0]:
pp.ProfileReport(data)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,44
Number of observations,5729
Total Missing (%),0.0%
Total size in memory,1.9 MiB
Average record size in memory,352.0 B
Numeric,15
Categorical,4
Boolean,0
Date,0
Text (Unique),1
Rejected,24


#1.2 Limpeza dos dados

In [0]:
y = data['Result']
print(y.shape)

(5729,)


### Excluindo colunas irrelevantes

In [0]:
data.drop('Result', axis=1, inplace=True)
data.drop('ID', axis=1,inplace=True)
data.drop('Date', axis=1,inplace=True)
data.drop('Year' , axis=1,inplace=True)

In [0]:
data_test.drop('Year' , axis=1,inplace=True)
data_test.drop('ID', axis=1,inplace=True)
data_test.drop('Date', axis=1,inplace=True)

### Merge dos dados TREINO e TESTE

In [0]:
merge_data = pd.concat((data, data_test)) 
merge_data.head()

,League,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR
0,Germany,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,8.0,3.0,13.0,8.0,2.0,0.0,8.0,3.0,13.0,8.0,2.0,0.0
1,Germany,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,9.0,4.0,14.0,0.0,2.0,0.0,9.0,4.0,14.0,0.0,2.0,0.0
2,Germany,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,14.0,8.0,6.0,4.0,0.0,0.0,14.0,8.0,6.0,4.0,0.0,0.0
3,Germany,Leverkusen,Freiburg,0.636943,0.250000,0.181818,0.617284,0.256410,0.200000,0.645161,0.270270,0.185185,0.636943,0.266667,0.166667,0.613497,0.238095,0.170940,0.636943,0.266667,0.166667,0.617284,0.238095,0.173913,1.69,1.62,2.44,2.25,19.0,10.0,22.0,7.0,2.0,0.0,19.0,10.0,22.0,7.0,2.0,0.0
4,Germany,Nurnberg,Dortmund,0.166667,0.250000,0.645161,0.166667,0.250000,0.653595,0.185185,0.270270,0.645161,0.166667,0.250000,0.653595,0.162075,0.235294,0.621118,0.161290,0.250000,0.653595,0.160000,0.238095,0.636943,1.83,1.76,2.16,2.07,18.0,8.0,20.0,6.0,2.0,0.0,18.0,8.0,20.0,6.0,2.0,0.0


In [0]:
attr_num = merge_data.dtypes[merge_data.dtypes != "object"].index
attr_num

Index(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
       'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH',
       'VCD', 'VCA', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'HS',
       'HST', 'HF', 'HC', 'HY', 'HR', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR'],
      dtype='object')

In [0]:
data.head()

,League,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR
0,Germany,Mainz,Greuther Furth,0.555556,0.285714,0.222222,0.534759,0.285714,0.230947,0.555556,0.294118,0.250000,0.546448,0.285714,0.230947,0.537634,0.270270,0.212766,0.546448,0.285714,0.230947,0.540541,0.270270,0.217391,2.09,1.98,1.94,1.83,8.0,3.0,13.0,8.0,2.0,0.0,8.0,3.0,13.0,8.0,2.0,0.0
1,Germany,Fortuna Dusseldorf,M'gladbach,0.344828,0.303030,0.416667,0.333333,0.303030,0.434783,0.344828,0.322581,0.434783,0.333333,0.294118,0.434783,0.312500,0.293255,0.414938,0.333333,0.303030,0.425532,0.312500,0.294118,0.416667,2.11,2.00,1.90,1.80,9.0,4.0,14.0,0.0,2.0,0.0,9.0,4.0,14.0,0.0,2.0,0.0
2,Germany,Hoffenheim,Ein Frankfurt,0.487805,0.294118,0.285714,0.487805,0.303030,0.263158,0.512821,0.303030,0.277778,0.500000,0.294118,0.266667,0.480769,0.277778,0.261780,0.487805,0.294118,0.285714,0.487805,0.277778,0.263158,2.00,1.93,1.95,1.88,14.0,8.0,6.0,4.0,0.0,0.0,14.0,8.0,6.0,4.0,0.0,0.0
3,Germany,Leverkusen,Freiburg,0.636943,0.250000,0.181818,0.617284,0.256410,0.200000,0.645161,0.270270,0.185185,0.636943,0.266667,0.166667,0.613497,0.238095,0.170940,0.636943,0.266667,0.166667,0.617284,0.238095,0.173913,1.69,1.62,2.44,2.25,19.0,10.0,22.0,7.0,2.0,0.0,19.0,10.0,22.0,7.0,2.0,0.0
4,Germany,Nurnberg,Dortmund,0.166667,0.250000,0.645161,0.166667,0.250000,0.653595,0.185185,0.270270,0.645161,0.166667,0.250000,0.653595,0.162075,0.235294,0.621118,0.161290,0.250000,0.653595,0.160000,0.238095,0.636943,1.83,1.76,2.16,2.07,18.0,8.0,20.0,6.0,2.0,0.0,18.0,8.0,20.0,6.0,2.0,0.0


# Normalização dos dados MERGE (TREINO  TESTE)

[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) é usado para padronizar 
* Calcule a média e o padrão para serem usados ​​em redimensionamentos posteriores.
* Executa a padronização centralizanda e dimensionando

In [0]:
from sklearn.preprocessing import StandardScaler

standardscaler = StandardScaler()
standardscaler = standardscaler.fit(data[attr_num])
normalizadas_train = standardscaler.transform(data[attr_num])
normalizadas_test = standardscaler.transform(data_test[attr_num])

normalizadas_train = pd.DataFrame(normalizadas_train)
normalizadas_test = pd.DataFrame(normalizadas_test)

normalizadas_train.columns = data[attr_num].columns
normalizadas_test.columns = data_test[attr_num].columns

normalizadas_train.index = data[attr_num].index
normalizadas_test.index = data_test[attr_num].index

In [0]:
data[attr_num] = normalizadas_train
data_test[attr_num] = normalizadas_test

In [0]:
data.head()

,League,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR
0,Germany,Mainz,Greuther Furth,0.409493,0.416812,-0.514809,0.296741,0.290607,-0.506843,0.381951,0.293838,-0.448155,0.357192,0.333780,-0.500761,0.366019,0.254176,-0.506711,0.399242,0.087126,-0.477476,0.372531,0.189578,-0.499214,0.425456,0.386324,-0.398741,-0.437366,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626
1,Germany,Fortuna Dusseldorf,M'gladbach,-0.683799,0.745503,0.615659,-0.763891,0.601905,0.691904,-0.753712,0.769943,0.652152,-0.771262,0.493893,0.698708,-0.800451,0.641953,0.660777,-0.731782,0.410484,0.679838,-0.810939,0.613308,0.660410,0.490359,0.456932,-0.474160,-0.503628,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626
2,Germany,Hoffenheim,Ein Frankfurt,0.057991,0.576324,-0.145677,0.049497,0.601905,-0.317411,0.151642,0.442921,-0.282750,0.111247,0.493893,-0.290569,0.071388,0.380835,-0.223668,0.088016,0.244050,-0.151741,0.098847,0.322975,-0.232889,0.133395,0.209806,-0.379886,-0.326930,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626
3,Germany,Leverkusen,Freiburg,0.831743,-0.261112,-0.749712,0.731284,-0.236204,-0.688840,0.864857,-0.105061,-0.834102,0.836366,-0.029144,-0.879018,0.759079,-0.288648,-0.748244,0.879506,-0.268568,-0.859789,0.770809,-0.382120,-0.752223,-0.872595,-0.884605,0.543997,0.490300,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626
4,Germany,Nurnberg,Dortmund,-1.608127,-0.261112,1.944089,-1.641494,-0.351444,1.978727,-1.614062,-0.105061,1.904874,-1.653771,-0.346702,1.986307,-1.579838,-0.335906,1.851413,-1.644833,-0.579800,2.036265,-1.602374,-0.382120,1.942241,-0.418277,-0.390355,0.016064,0.092729,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626


In [0]:
df = pd.concat((data, data_test))
df.head()

,League,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR
0,Germany,Mainz,Greuther Furth,0.409493,0.416812,-0.514809,0.296741,0.290607,-0.506843,0.381951,0.293838,-0.448155,0.357192,0.333780,-0.500761,0.366019,0.254176,-0.506711,0.399242,0.087126,-0.477476,0.372531,0.189578,-0.499214,0.425456,0.386324,-0.398741,-0.437366,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626
1,Germany,Fortuna Dusseldorf,M'gladbach,-0.683799,0.745503,0.615659,-0.763891,0.601905,0.691904,-0.753712,0.769943,0.652152,-0.771262,0.493893,0.698708,-0.800451,0.641953,0.660777,-0.731782,0.410484,0.679838,-0.810939,0.613308,0.660410,0.490359,0.456932,-0.474160,-0.503628,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626
2,Germany,Hoffenheim,Ein Frankfurt,0.057991,0.576324,-0.145677,0.049497,0.601905,-0.317411,0.151642,0.442921,-0.282750,0.111247,0.493893,-0.290569,0.071388,0.380835,-0.223668,0.088016,0.244050,-0.151741,0.098847,0.322975,-0.232889,0.133395,0.209806,-0.379886,-0.326930,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626
3,Germany,Leverkusen,Freiburg,0.831743,-0.261112,-0.749712,0.731284,-0.236204,-0.688840,0.864857,-0.105061,-0.834102,0.836366,-0.029144,-0.879018,0.759079,-0.288648,-0.748244,0.879506,-0.268568,-0.859789,0.770809,-0.382120,-0.752223,-0.872595,-0.884605,0.543997,0.490300,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626
4,Germany,Nurnberg,Dortmund,-1.608127,-0.261112,1.944089,-1.641494,-0.351444,1.978727,-1.614062,-0.105061,1.904874,-1.653771,-0.346702,1.986307,-1.579838,-0.335906,1.851413,-1.644833,-0.579800,2.036265,-1.602374,-0.382120,1.942241,-0.418277,-0.390355,0.016064,0.092729,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626


In [0]:
data.head()

,League,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,HS,HST,HF,HC,HY,HR,AS,AST,AF,AC,AY,AR
0,Germany,Mainz,Greuther Furth,0.409493,0.416812,-0.514809,0.296741,0.290607,-0.506843,0.381951,0.293838,-0.448155,0.357192,0.333780,-0.500761,0.366019,0.254176,-0.506711,0.399242,0.087126,-0.477476,0.372531,0.189578,-0.499214,0.425456,0.386324,-0.398741,-0.437366,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626,-1.375475,-0.787639,-0.212642,1.819262,-0.080507,-0.131626
1,Germany,Fortuna Dusseldorf,M'gladbach,-0.683799,0.745503,0.615659,-0.763891,0.601905,0.691904,-0.753712,0.769943,0.652152,-0.771262,0.493893,0.698708,-0.800451,0.641953,0.660777,-0.731782,0.410484,0.679838,-0.810939,0.613308,0.660410,0.490359,0.456932,-0.474160,-0.503628,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626,-1.063089,-0.213909,0.107186,-2.937315,-0.080507,-0.131626
2,Germany,Hoffenheim,Ein Frankfurt,0.057991,0.576324,-0.145677,0.049497,0.601905,-0.317411,0.151642,0.442921,-0.282750,0.111247,0.493893,-0.290569,0.071388,0.380835,-0.223668,0.088016,0.244050,-0.151741,0.098847,0.322975,-0.232889,0.133395,0.209806,-0.379886,-0.326930,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626,0.498842,2.081009,-2.451436,-0.559026,-2.261234,-0.131626
3,Germany,Leverkusen,Freiburg,0.831743,-0.261112,-0.749712,0.731284,-0.236204,-0.688840,0.864857,-0.105061,-0.834102,0.836366,-0.029144,-0.879018,0.759079,-0.288648,-0.748244,0.879506,-0.268568,-0.859789,0.770809,-0.382120,-0.752223,-0.872595,-0.884605,0.543997,0.490300,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626,2.060772,3.228468,2.665808,1.224690,-0.080507,-0.131626
4,Germany,Nurnberg,Dortmund,-1.608127,-0.261112,1.944089,-1.641494,-0.351444,1.978727,-1.614062,-0.105061,1.904874,-1.653771,-0.346702,1.986307,-1.579838,-0.335906,1.851413,-1.644833,-0.579800,2.036265,-1.602374,-0.382120,1.942241,-0.418277,-0.390355,0.016064,0.092729,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626,1.748386,2.081009,2.026153,0.630118,-0.080507,-0.131626


In [0]:
data['train'] = 1
data_test['train'] = 0

merge_data = pd.concat((data, data_test))

In [0]:
merge_data = pd.get_dummies(merge_data)

### Separando os dados do merge

In [0]:
data = merge_data.loc[merge_data['train'] == 1]
data_test = merge_data.loc[merge_data['train'] == 0]

### Excluindo coluna de verificação

In [0]:
data.drop('train', axis=1, inplace=True)
data_test.drop('train', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
sns.countplot(y,facecolor=(0, 0, 0, 0),linewidth=5,edgecolor=sns.color_palette("dark", 3))

### Igualando os dados

In [0]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
df_balanced, result_balanced = ros.fit_sample(data, y)

In [0]:
sns.countplot(result_balanced,
                    facecolor=(0, 0, 0, 0),
                   linewidth=5,
                    edgecolor=sns.color_palette("dark", 3))

In [0]:
#kfold = StratifiedKFold(n_splits=30, shuffle=True, random_state=100)
kfold  = StratifiedKFold(n_splits=5, shuffle=True, random_state=3058)
kfold

StratifiedKFold(n_splits=5, random_state=3058, shuffle=True)

# Função para mostrar os resultados dos modelos

In [0]:
def cross_val(model, X_train, Y_train, kfold):
    acc = cross_val_score(model, X_train, Y_train, cv=kfold)
    prec = cross_val_score(model, X_train, Y_train, scoring='precision_macro', cv=kfold)
    rec = cross_val_score(model, X_train, Y_train, scoring='recall_macro', cv=kfold)
    f1 = cross_val_score(model, X_train, Y_train, scoring='f1_macro', cv=kfold)
    
    print("Mediana: \n")
    print("Accuracy: %.2f%% (%.2f%%)" % (np.ma.median(acc) * 100, acc.std() * 100))
    print("Precision: %.2f%% (%.2f%%)" % (np.ma.median(prec) * 100, prec.std() * 100))
    print("Recall: %.2f%% (%.2f%%)" % (np.ma.median(rec) * 100, rec.std() * 100))
    print("F-measure: %.2f%%, (%.2f%%)" % (np.ma.median(f1) * 100,f1.std() * 100))
    print('')
    print("Média: \n")
    print("Accuracy: %.2f%% (%.2f%%)" % (acc.mean() * 100, acc.std() * 100))
    print("Precision: %.2f%% (%.2f%%)" % (prec.mean() * 100, prec.std() * 100))
    print("Recall: %.2f%% (%.2f%%)" % (rec.mean() * 100, rec.std() * 100))
    print("F-measure: %.2f%%, (%.2f%%)" % (f1.mean() * 100,f1.std() * 100))

# Bibliotecas dos modelos 

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold

# Modelo : [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) selecionado para CV

Os parâmetros selecionados são aqueles que maximizam a pontuação dos dados deixados de fora, a menos que uma pontuação explícita seja passada e, nesse caso, ela será usada.

# Modelo : Regression Logistic

In [0]:
model = LogisticRegression()
cross_val(model , df_balanced, result_balanced , kfold)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

Mediana: 

Accuracy: 48.18% (1.04%)
Precision: 48.08% (1.04%)
Recall: 48.18% (1.04%)
F-measure: 48.13%, (1.06%)

Média: 

Accuracy: 48.11% (1.04%)
Precision: 48.11% (1.04%)
Recall: 48.11% (1.04%)
F-measure: 48.09%, (1.06%)


# Modelo : Regression CV

In [0]:
grid={"C":np.logspace(0, 4, 10), "penalty":["l1","l2"]}# l1 lasso l2 ridge

logreg_cv= GridSearchCV(LogisticRegression() , grid , cv=kfold, scoring=make_scorer(f1_score, average='macro'))
logreg_cv.fit(df_balanced,result_balanced)
# View best hyperparameters
print(logreg_cv.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

{'C': 1.0, 'penalty': 'l1'}


In [0]:
print(logreg_cv.best_params_)

{'C': 1.0, 'penalty': 'l1'}


In [0]:
model = LogisticRegression(penalty='l1')
cross_val(model , df_balanced, result_balanced , kfold)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

Mediana: 

Accuracy: 48.44% (1.36%)
Precision: 48.50% (1.36%)
Recall: 48.50% (1.34%)
F-measure: 48.43%, (1.32%)

Média: 

Accuracy: 48.56% (1.36%)
Precision: 48.61% (1.36%)
Recall: 48.56% (1.34%)
F-measure: 48.54%, (1.32%)


# Modelo : DecisionTreeClassifier

In [0]:
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
cross_val(DecisionTreeClassifier(),df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 67.88% (1.31%)
Precision: 67.28% (1.35%)
Recall: 67.18% (1.24%)
F-measure: 66.97%, (1.25%)

Média: 

Accuracy: 67.60% (1.31%)
Precision: 67.27% (1.35%)
Recall: 67.06% (1.24%)
F-measure: 66.89%, (1.25%)


# Modelo : DecisionTreeClassifier CV

In [0]:
tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,25,30,40,50,70,90,120,150]}
clf = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=kfold,scoring='accuracy')
clf.fit(df_balanced,result_balanced)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=3058, shuffle=True),
             error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
     

In [0]:
print(clf.best_params_)

{'criterion': 'gini', 'max_depth': 120}


In [0]:
cross_val(DecisionTreeClassifier(max_depth=120),df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 67.69% (1.30%)
Precision: 66.72% (0.99%)
Recall: 67.69% (1.37%)
F-measure: 66.69%, (1.45%)

Média: 

Accuracy: 67.18% (1.30%)
Precision: 67.31% (0.99%)
Recall: 67.48% (1.37%)
F-measure: 66.79%, (1.45%)


# Modelo : KNN

In [0]:
cross_val(KNeighborsClassifier(),df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 51.24% (1.16%)
Precision: 52.24% (1.06%)
Recall: 51.24% (1.16%)
F-measure: 51.01%, (1.14%)

Média: 

Accuracy: 51.24% (1.16%)
Precision: 52.15% (1.06%)
Recall: 51.24% (1.16%)
F-measure: 51.07%, (1.14%)


# Modelo : KNN CV

In [0]:
k_range = list(range(1, 31))
weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors = k_range, weights = weight_options)
# instantiate the grid
gridKNN = GridSearchCV(knn, param_grid, cv=kfold,scoring=make_scorer(f1_score, average='macro'))
gridKNN.fit(X_train, y_train)

In [0]:
#gridKNN.best_params_

# Modelo : AdaBoostClassifier

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
model_adaboost = AdaBoostClassifier()
cross_val(model_adaboost,df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 47.99% (0.59%)
Precision: 49.36% (0.78%)
Recall: 47.99% (0.59%)
F-measure: 48.11%, (0.53%)

Média: 

Accuracy: 48.15% (0.59%)
Precision: 49.49% (0.78%)
Recall: 48.15% (0.59%)
F-measure: 48.32%, (0.53%)


In [0]:
model_adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=25),
                              algorithm="SAMME", n_estimators=25)
cross_val(model_adaboost,df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 71.51% (0.12%)
Precision: 71.91% (1.15%)
Recall: 71.78% (0.73%)
F-measure: 71.34%, (0.59%)

Média: 

Accuracy: 71.46% (0.12%)
Precision: 71.73% (1.15%)
Recall: 72.11% (0.73%)
F-measure: 71.46%, (0.59%)


# Modelo : AdaBoostClassifier CV

In [0]:
params = {'base_estimator__max_depth':[1,50],
          'base_estimator':[DecisionTreeClassifier(max_depth=120), 
                            DecisionTreeClassifier(max_features=25)]}
gs = GridSearchCV(AdaBoostClassifier(base_estimator=DecisionTreeClassifier()), params)
gs.fit(df_balanced,result_balanced)
print(gs.best_estimator_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=None,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
                                                                           

In [0]:
print(gs.best_estimator_)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=50,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                            

In [0]:
model_adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=25),
                              algorithm="SAMME", n_estimators=300)
cross_val(model_adaboost,df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 73.31% (1.25%)
Precision: 74.18% (0.74%)
Recall: 73.42% (0.78%)
F-measure: 73.06%, (0.87%)

Média: 

Accuracy: 73.31% (1.25%)
Precision: 73.87% (0.74%)
Recall: 73.96% (0.78%)
F-measure: 73.46%, (0.87%)


In [0]:
model_adaboost.fit(df_balanced, result_balanced)

AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=25,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                              

In [0]:
adaboost_predict = model_adaboost.predict(data_test)

In [0]:
adaboost_predict

In [0]:
result = pd.read_csv('sample_submission.csv')
result.drop('Result', axis=1, inplace=True)

In [0]:
result.head()

,ID
0,GER090916#0
1,GER100916#0
2,GER100916#1
3,GER100916#2
4,GER100916#3


In [0]:
result['Category'] = adaboost_predict
result.to_csv('adaboost_pre.csv', index=False)
result

,ID,Category
0,GER090916#0,0
1,GER100916#0,0
2,GER100916#1,0
3,GER100916#2,0
4,GER100916#3,2
...,...,...
2843,SPA190518#5,0
2844,SPA200518#0,2
2845,SPA200518#1,2
2846,SPA200518#2,2


# Modelo GaussianNB

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
GNB_clf = GaussianNB()
cross_val(GNB_clf,df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 47.93% (0.99%)
Precision: 47.22% (1.17%)
Recall: 47.93% (0.99%)
F-measure: 47.15%, (1.08%)

Média: 

Accuracy: 48.07% (0.99%)
Precision: 47.37% (1.17%)
Recall: 48.07% (0.99%)
F-measure: 47.20%, (1.08%)


# Modelo GradientBoostingClassifier

In [0]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [0]:
gb_params = {
    'n_estimators' : [100,200,300],
    'learning_rate' : [.1,.2,.3],
    'max_depth' : [3,5,7]
}

gb_class = GradientBoostingClassifier()

gb_grid = GridSearchCV(gb_class, gb_params, cv = 5, n_jobs=-1)
gb_grid.fit(df_balanced, result_balanced)

gb_grid.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=7,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
gb_opt = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=7,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
cross_val(gb_opt,df_balanced,result_balanced,kfold)

Mediana: 

Accuracy: 69.79% (0.93%)
Precision: 70.01% (0.48%)
Recall: 70.24% (0.92%)
F-measure: 70.48%, (0.96%)

Média: 

Accuracy: 70.05% (0.93%)
Precision: 70.04% (0.48%)
Recall: 70.68% (0.92%)
F-measure: 70.60%, (0.96%)
